# Data Structures and Algorithms in Python - Ch. 11: Binary Search Trees
### AJ Zerouali, 2023/10/08

## 0) Introduction

**References:**

- Chapter 11 of "Data structures and algorithms in Python", by Goodrich, Tamassia and Goldwasser (primary, abbreviated [GTG13]). 
- Section 16 of "Python for Data Structures, Algorithms, and Interviews!" by Jose Portilla (to a much lesser extent).

**Comments:**
- Heaps were used to implement priority queues. Binary search trees seem to be particularly useful for the implementation of *sorted maps*, as described in sections 10.1 and 10.3 of [GTG13]. In plain terms, a map is essentially a Python dictionary: It stores values associated to a set of keys.
- I am having the same issues as with parts A and B here: [GTG13] make it impossible to come-up with a straightforward implementation of binary search trees. It's not that complicated, but they absolutely have to involve every previous chapter of their book in their implementations. The good think is that the previous data structures are actually used in practice, but it's still over-complicated.
- It's unclear how 

## 1) Basics of binary search trees

This section is based on [GTG13, Sec.11.1, 11.2].

### 1.a - Binary search trees

#### Definition:
A **binary search tree** (BST) is a binary tree $T$ whose nodes store key-value pairs, and such that for each position $p$ in $T$ containing $(k,v)$:
1) Keys stored in the left subtree of $p$ are lower than $k$.
2) Keys stored in the right subtree of $p$ are greater $k$.

As one would expect, this data structure simplifies the retrieval of a specific key within its nodes, and easily lends itself to a recursive implementation of a *find()* function that either returns the current position or visits the subtree of lower/larger keys. Intuitively, we expect the search to be linear in the height of the tree. An important observation is that based on this structure, adding a node to the tree will also require a search for the correct position, as we shall see below. On this topic, the following proposition will become more relevant:

#### Proposition 11.1:
An inorder traversal of a binary search tree (BST) visits positions in increasing order of their keys.

#### Proof sketch:
This is easy to see by induction on the height $h$ of the BST. For $h=0$, there is nothing to prove, for $h=1$, the order of visit is left child, root, right child, which is precisely the increasing order of the keys (provided the existence of children). By depicting a tree of height $h=2$, we again see that the proposition holds true.

Regarding the induction step, suppose the proposition is true up to a fixed height $h$, and consider a tree of height $h+1$, consisting of a root $\rho$, a left subtree $T_L$ of height $h_L\le h$, and a right subtree $T_R$ of height $h_R\le h$. Again, the order of visit is $T_L$, $\rho$, and then $T_R$, which is again preserving the increasing order of all the keys in $T$.

QED

### 1.b - Balanced (binary) search trees

The remainder of chapter 11 discusses several types of BSTs, including:
- AVL trees (Sec.11.3)
- Splay trees (Sec.11.4)
- Red-black trees (Sec.11.6)
- (2,4) trees (Sec.11.5)

The first three of the above are **balanced search trees**, which are the subject of section 11.2. We will also breifly discuss balanced search trees here.

#### Definitions:
Consider a BST $T$.
- A **rotation** on a position $p$ and its left child $l_p$ consists of reshaping the subtree of $T$ rooted at $p$ as follows: If $T_1$ and $T_2$ are the left and right subtrees of $l_c$ and $T_3$ is the right subtree of $p$, we *rotate* $l_p$ to $p$ by setting $p$ to be the left child of $l_p$, and set $T_2$ and $T_3$ to be the new left and right subtrees of $p$ ($T_1$ remains the left subtree of $l_p$, see [GTG13, Fig.11.8]).
- A **trinode restructuring** is when we have a position $x$, its parent $y$, and its grandparent $z$, and depending on the left/right positions of $x$ and $y$, we perform 1 or 2 rotations with $z$ and/or $y$ to lower the height of the tree. This is depicted in [GTG13, Sec.11.9].

The purpose of rebalancing a tree is to ensure that it has the lowest possible height, which will in turn give guarantees on the execution time of the search and addition methods. 

### 1.c - Searching in a binary search tree

This part is based on [GTG13, Sec.11.1.2].

By Proposition 11.1, we know that an inorder traversal will visit the positions of a BST in increasing order of the keys. This will be used to locate the position corresponding to a key, to add a key-value node to the BST, as well as determine the largest previous key of a position. The pseudo-code for the search method is as follows:

        Algorithm BSTSearch(T,p,k):
            '''
                Params
                T: Tree on which to perform the search
                p: Start position of the search
                k: Key to search for
            '''
            # Search in left subtree
            if k<p.key() and T.left(p) != None:
                return BSTSearch(T, T.left(p), k)
            
            # Successful search
            elif k==p.key():
                return p
            
            # Search in right subtree
            elif k>p.key() and T.right(p) != None:
                return BSTSearch(T, T.left(p), k)
                
            # Unsuccessful search
            return p
            
An important comment at this point which will be important later is that for unsuccessful searches, *BSTSearch()* always returns the last position visited. To avoid any issues, this implies that we should always check the key *k* after a call to this algorithm, and although this looks cumbersome at first glance, it is useful for insertions.

Now we analyze the worst-case running time of this algorithm, assuming that we start with *p* being the root of *T*. Each call to *BSTSearch()* executes a constant number of comparisons before the *return* statement. On the other hand, the worst cases are encountered when *k* is lower/greater than all keys in *T*, or when *k* is stored in a leaf.

We in fact spend an $O(1)$ amount of time at each level of the tree, and in the worst cases we make $h$ comparisons, where $h$ is the height of *T*. In conclusion, *BSTSearch()* has $O(h)$ time complexity, with $\log(n+1)-1\le h \le n-1.$.

An important remark at this point is the purpose of using balanced search trees: The idea is to ensure that $O(h)=O(\log n)$. Without rebalancing, it is easy to see that a linked list consisting of strictly increasing keys can be a BST, in which case $h=n$. We will later see how to avoid this situation.

### 1.d - Insertion and deletion

This subsection is based on [GTG13, Sec.11.1.3]. 

We start with insertions of a key-value item. The main point here is that any new node should be added to the appropriate position in the BST, meaning that the *T.add(k,v)* call should call the *BSTSearch(T, T.root(), k)* algorithm previously discussed. If the key *k* is found in *T*, then *v* becomes the new assigned value, and if the key *k* should be added right after the greatest key that is lower than *k*. As a first model, our insertion algorithm should thus be:

        Algorithm BSTInsert(T, k, v):
            '''
                Implementation of the call
                M[k] = v
                for a sorted map M based on a BST.
            '''
            p = BSTSearch(T, T.root(), k)
            
            # Key k was found in T
            if k == p.key():
                Set p's value to v
                
            # Key k was not found
            elif k < p.key():
                Add node with item (k,v) as left child of p
            elif k > p.key():
                Add node with item (k,v) as right child of p

In practice, we will add rebalancing instructions to ensure the tree is balanced.

The hard operation here is deletion, as one would expect. Assuming the key *k* is found in the tree at position *P*, there are 3 cases to consider:
1) The position *p* is a leaf: Here we simply delete this position.
2) The position *p* has only one child: In which case *p* is simply replaced by its only existing child.
3) The position *p* has two children: For this case there are 3 steps.
    - Locate the position *o=before(p)* with greatest key strictly lower than *p* (this will be the rightmost position in the left subtree of *p*).
    - Replace the node at position *p* by the node at position *r*. 
    - Delete the node at position *r* from *T*.
    
    With this procedure, we preserve the BST property. For the second step, it's important to note that *r* cannot have a right child, meaning that it has at most one child, and the process of case (2) is repeated.

## 2) Implementation of a BST

### 2.a - Preliminaries

We will need two abstract base classes. The first one is the *map* ADT, and we use the implementation discussed in [GTG13, Sec.10.1.4] (source: https://github.com/mjwestcott/Goodrich/blob/master/ch10/map_base.py).

In [1]:
#from collections import MutableMapping
from collections.abc import MutableMapping

class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""
    #------------------------------- nested _Item class -------------------------------
    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key   # compare items based on their keys

        def __ne__(self, other):
            return not (self == other)       # opposite of __eq__

        def __lt__(self, other):
            return self._key < other._key    # compare items based on their keys

In the above, it's important to note that the *MutableMapping* class has the following abstract methods:

    __getitem__(self, k), __setitem__(self, k, v), __delitem__(self, k), __iter__(self), __len__(self)

which respectively stand for *M[k]*, *M[k]=v*, *del M[k]*, *iter(M)* and *len(M)*. We will implement these methods below, with the most involved parts being the *__setitem__()* and *__delitem__()* methods.


The second one is the linked binary tree class of [GTG13, Sec.8.3.1]. The source code for this part is divided accross several files:
* Linked queue: https://github.com/mjwestcott/Goodrich/blob/master/ch07/linked_queue.py
* Tree: https://github.com/mjwestcott/Goodrich/blob/master/ch08/tree.py
* Binary tree: https://github.com/mjwestcott/Goodrich/blob/master/ch08/binary_tree.py
* Linked binary tree: https://github.com/mjwestcott/Goodrich/blob/master/ch08/linked_binary_tree.py

In [2]:
from Code.Trees.linked_binary_tree import LinkedBinaryTree

#### Temp

In [2]:
#from ..exceptions import Empty # I'm unable to locate this submodule in this repo.
import collections

'''
    #### LINKED QUEUE ####
'''
class LinkedQueue:
    """FIFO queue implementation using a singly linked list for storage."""
    
    #-------------------------- nested _Node class --------------------------
    class _Node:
        """Lightweight, nonpublic class for storing a singly linked node."""
        __slots__ = '_element', '_next'         # streamline memory usage
        
        def __init__(self, element, next):
            self._element = element
            self._next = next

    #------------------------------- queue methods -------------------------------
    def __init__(self):
        """Create an empty queue."""
        self._head = None
        self._tail = None
        self._size = 0                          # number of queue elements

    def __len__(self):
        """Return the number of elements in the queue."""
        return self._size

    def is_empty(self):
        """Return True if the queue is empty."""
        return self._size == 0

    def first(self):
        """Return (but do not remove) the element at the front of the queue.
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise ValueError('Queue is empty') #Empty('Queue is empty')
        return self._head._element              # front aligned with head of list

    def dequeue(self):
        """Remove and return the first element of the queue (i.e., FIFO).
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise ValueError('Queue is empty') #Empty('Queue is empty')
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():                     # special case as queue is empty
            self._tail = None                     # removed head had been the tail
        return answer

    def enqueue(self, e):
        """Add an element to the back of queue."""
        newest = self._Node(e, None)            # node will be new tail node
        if self.is_empty():
            self._head = newest                   # special case: previously empty
        else:
            self._tail._next = newest
        self._tail = newest                     # update reference to tail node
        self._size += 1

In [5]:
'''
    #### BASE TREE ####
'''
class Tree:
    """Abstract base class representing a tree structure."""
    #------------------------------- nested Position class -------------------------------
    class Position:
        """An abstraction representing the location of a single element within a tree.
        Note that two position instaces may represent the same inherent location in a tree.
        Therefore, users should always rely on syntax 'p == q' rather than 'p is q' when testing
        equivalence of positions.
        """

        def element(self):
            """Return the element stored at this Position."""
            raise NotImplementedError('must be implemented by subclass')
      
        def __eq__(self, other):
            """Return True if other Position represents the same location."""
            raise NotImplementedError('must be implemented by subclass')

        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not (self == other)            # opposite of __eq__

    # ---------- abstract methods that concrete subclass must support ----------
    def root(self):
        """Return Position representing the tree's root (or None if empty)."""
        raise NotImplementedError('must be implemented by subclass')

    def parent(self, p):
        """Return Position representing p's parent (or None if p is root)."""
        raise NotImplementedError('must be implemented by subclass')

    def num_children(self, p):
        """Return the number of children that Position p has."""
        raise NotImplementedError('must be implemented by subclass')

    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        raise NotImplementedError('must be implemented by subclass')

    def __len__(self):
        """Return the total number of elements in the tree."""
        raise NotImplementedError('must be implemented by subclass')

    # ---------- concrete methods implemented in this class ----------
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root() == p

    def is_leaf(self, p):
        """Return True if Position p does not have any children."""
        return self.num_children(p) == 0

    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0

    def depth(self, p):
        """Return the number of levels separating Position p from the root."""
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _height1(self):                 # works, but O(n^2) worst-case time
        """Return the height of the tree."""
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

    def _height2(self, p):                  # time is linear in size of subtree
        """Return the height of the subtree rooted at Position p."""
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))

    def height(self, p=None):
        """Return the height of the subtree rooted at Position p.

        If p is None, return the height of the entire tree.
        """
        if p is None:
            p = self.root()
        return self._height2(p)        # start _height2 recursion

    def __iter__(self):
        """Generate an iteration of the tree's elements."""
        for p in self.positions():                        # use same order as positions()
            yield p.element()                               # but yield each element

    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.preorder()                            # return entire preorder iteration

    def preorder(self):
        """Generate a preorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):  # start recursion
                yield p

    def _subtree_preorder(self, p):
        """Generate a preorder iteration of positions in subtree rooted at p."""
        yield p                                           # visit p before its subtrees
        for c in self.children(p):                        # for each child c
            for other in self._subtree_preorder(c):         # do preorder of c's subtree
                yield other                                   # yielding each to our caller

    def postorder(self):
        """Generate a postorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):  # start recursion
                yield p

    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):                        # for each child c
            for other in self._subtree_postorder(c):        # do postorder of c's subtree
                yield other                                   # yielding each to our caller
        yield p                                           # visit p after its subtrees

    def breadthfirst(self):
        """Generate a breadth-first iteration of the positions of the tree."""
        if not self.is_empty():
            fringe = LinkedQueue()             # known positions not yet yielded
            fringe.enqueue(self.root())        # starting with the root
            while not fringe.is_empty():
                p = fringe.dequeue()             # remove from front of the queue
                yield p                          # report this position
                for c in self.children(p):
                    fringe.enqueue(c)              # add children to back of queue

In [ ]:
'''
    #### BASE BINARY TREE ####
'''
class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""

    # --------------------- additional abstract methods ---------------------
    def left(self, p):
        """Return a Position representing p's left child.

        Return None if p does not have a left child.
        """
        raise NotImplementedError('must be implemented by subclass')

    def right(self, p):
        """Return a Position representing p's right child.

        Return None if p does not have a right child.
        """
        raise NotImplementedError('must be implemented by subclass')

    # ---------- concrete methods implemented in this class ----------
    def sibling(self, p):
        """Return a Position representing p's sibling (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:                    # p must be the root
            return None                         # root has no sibling
        else:
            if p == self.left(parent):
                return self.right(parent)         # possibly None
            else:
                return self.left(parent)          # possibly None

    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

    def inorder(self):
        """Generate an inorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p

    def _subtree_inorder(self, p):
        """Generate an inorder iteration of positions in subtree rooted at p."""
        if self.left(p) is not None:          # if left child exists, traverse its subtree
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p                               # visit p between its subtrees
        if self.right(p) is not None:         # if right child exists, traverse its subtree
            for other in self._subtree_inorder(self.right(p)):
                yield other

    # override inherited version to make inorder the default
    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.inorder()                 # make inorder the default

In [ ]:
'''
    #### LINKED BINARY TREE ####
'''
class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure."""

    #-------------------------- nested _Node class --------------------------
    class _Node:
        """Lightweight, nonpublic class for storing a node."""
        __slots__ = '_element', '_parent', '_left', '_right' # streamline memory usage

    def __init__(self, element, parent=None, left=None, right=None):
        self._element = element
        self._parent = parent
        self._left = left
        self._right = right

    #-------------------------- nested Position class --------------------------
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element."""

        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node

        def element(self):
            """Return the element stored at this Position."""
            return self._node._element

        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node

    #------------------------------- utility methods -------------------------------
    def _validate(self, p):
        """Return associated node, if position is valid."""
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:      # convention for deprecated nodes
            raise ValueError('p is no longer valid')
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node (or None if no node)."""
        return self.Position(self, node) if node is not None else None

    #-------------------------- binary tree constructor --------------------------
    def __init__(self):
        """Create an initially empty binary tree."""
        self._root = None
        self._size = 0

    #-------------------------- public accessors --------------------------
    def __len__(self):
        """Return the total number of elements in the tree."""
        return self._size
  
    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)

    def parent(self, p):
        """Return the Position of p's parent (or None if p is root)."""
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        """Return the Position of p's left child (or None if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        """Return the Position of p's right child (or None if no right child)."""
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        if node._left is not None:     # left child exists
            count += 1
        if node._right is not None:    # right child exists
            count += 1
        return count

    #-------------------------- nonpublic mutators --------------------------
    def _add_root(self, e):
        """Place element e at the root of an empty tree and return new Position.

        Raise ValueError if tree nonempty.
        """
        if self._root is not None:
            raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        """Create a new left child for Position p, storing element e.

        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a left child.
        """
        node = self._validate(p)
        if node._left is not None:
            raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)                  # node is its parent
        return self._make_position(node._left)

    def _add_right(self, p, e):
        """Create a new right child for Position p, storing element e.

        Return the Position of new node.
        Raise ValueError if Position p is invalid or p already has a right child.
        """
        node = self._validate(p)
        if node._right is not None:
            raise ValueError('Right child exists')
        self._size += 1
        node._right = self._Node(e, node)                 # node is its parent
        return self._make_position(node._right)

    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self, p):
        """Delete the node at Position p, and replace it with its child, if any.

        Return the element that had been stored at Position p.
        Raise ValueError if Position p is invalid or p has two children.
        """
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError('Position has two children')
        child = node._left if node._left else node._right  # might be None
        if child is not None:
            child._parent = node._parent   # child's grandparent becomes parent
        if node is self._root:
            self._root = child             # child becomes root
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node              # convention for deprecated node
        return node._element
  
    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2, respectively, as the left and right subtrees of the external Position p.

        As a side effect, set t1 and t2 to empty.
        Raise TypeError if trees t1 and t2 do not match type of this tree.
        Raise ValueError if Position p is invalid or not external.
        """
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2):    # all 3 trees must be same type
            raise TypeError('Tree types must match')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():         # attached t1 as left subtree of node
            t1._root._parent = node
            node._left = t1._root
            t1._root = None             # set t1 instance to empty
            t1._size = 0
        if not t2.is_empty():         # attached t2 as right subtree of node
            t2._root._parent = node
            node._right = t2._root
            t2._root = None             # set t2 instance to empty
            t2._size = 0

### 2.b - 

In [7]:
class BSTMap(LinkedBinaryTree, MapBase):
    '''
        Implementation of a map as a
        binary search tree.
        No rebalancing done in this version.
    '''
    #---------- Nested Position class ----------
    class Position(LinkedBinaryTree.Position):
        '''
            Abstract position class for nodes
            of BST
        '''
        def key(self):
            '''
                Return key attribute of item contained in current position
            '''
            return self.element()._key
        
        def value(self):
            '''
                Return value attribute of item contained in current position
            '''
            return self.element()._value
    
    #---------- Private tree search methods ----------
    def _search_subtree(self, p, k):
        '''
            Search for key k in subtree
            rooted at position p.
            Assumes tree is non-empty.
        '''
        if k<p.key() and self.left(p) is not None:
            return self._search_subtree(self.left(p), k)
        elif k==p.key():
            return p
        elif k>p.key() and self.right(p) is not None:
            return self._search_subtree(self.right(p), k)
        return p
    
    def _subtree_first_position(self, p):
        '''
            Return position of first item
            in subtree rooted at p.
        '''
        cur_pos = p
        while self.left(cur_pos) is not None:
            cur_pos = self.left(cur_pos)
        return cur_pos
    
    def _subtree_last_position(self, p):
        '''
            Return position of last item
            in subtree rooted at p.
        '''
        cur_pos = p
        while self.right(cur_pos) is not None:
            cur_pos = self.right(cur_pos)
        return cur_pos
            
    #---------- Private tree restructuring methods ----------
    '''
        To be implemented in subclasses of BSTMap()
    '''
    def _rebalance_insert(self, p):
        pass
    def _rebalance_delete(self, p):
        pass
    def _rebalance_access(self, p):
        pass
    
    def _relink(self, p):
        pass
    
    def _rotate(self, p):
        pass
    
    def _trinode_restructure(self, p):
        pass
    #---------- Public Map methods ----------
    def __getitem__(self, k):
        '''
            Search for key k in map and
            return associated value.
            Implements the call M[k]
        '''
        if self.is_empty():
            raise ValueError("Map is empty")
        p = self._search_subtree(self.root(), k)
        if k==p.key():
            return p.value()
        else:
            raise ValueError(f"Key \'{repr(k)}\' not found")
    
    def __iter__(self):
        '''
            Generate iterator over all keys in increasing order.
        '''
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)
    
    def __setitem__(self, k, v):
        '''
            Assign value v to key k.
        '''
        # If tree is empty, add (k,v) as root
        if self.is_empty():
            root = self._add_root(self._Item(k,v))
            self._rebalance_insert(root)
        # If tree is not empty, search for key
        else:
            p = self._search_subtree(self.root(), k)
            
            # If k already exists in tree, replace value
            if k == p.key():
                p.element()._value = v
                self._rebalance_access(p)
                return
            # If k doesn't exist, add node containing (k,v)
            else:
                if k<p.key():
                    node = self._add_left(p, self._Item(k,v))
                elif k>p.key():
                    node = self._add_right(p, self._Item(k,v))
                
                # Rebalance at node
                self._rebalance_insert(node)
    
    def delete(self, p):
        '''
            Delete item at position p and
            reshape tree to preserve BST
            property. Calls:
            - LinkedBinaryTree._delete(Position).
            - LinkedBinaryTree._replace(Position1, Position2).
        '''
        self._validate(p)
        
        # Case where p has two children
        if (self.left(p) is not None) and (self.right(p) is not None):
            replacement_pos = self._subtree_last_position(self.left(p))
            self._replace(p, replacement_pos)
            p = replacement_pos
        
        # Case where p (now) has at most one child
        parent = self.parent(p)
        self._delete(p)
        self._rebalance_delete(parent)
    
    def __delitem__(self, k):
        '''
            Delete item with key k from
            map. Calls delete(Position)
            method.
        '''
        if self.is_empty():
            raise ValueError("Map is empty.")
        else:
            # BST search for node containing key k
            p = self._search_subtree(self.root(), k)
            if k == p.key():
                # Call delete position method
                self.delete(p)
                return
            else:
                self._rebalance_access(p)
                raise ValueError(f"Key \'{repr(k)}\' not found")
    
    #---------- Public tree navigation methods ----------
    
    def first(self):
        '''
            Return first Position in tree
            (i.e. position of lowest key)
        '''
        # Empty tree case
        if len(self) == 0:
            return None
        else:
            return self._subtree_first_position(self.root())
    
    def last(self):
        '''
            Return last Position in tree
            (i.e. position of greatest key)
        '''
        # Empty tree case
        if len(self) == 0:
            return None
        else:
            return self._subtree_last_position(self.root())
    
    def before(self, p):
        '''
            Return Position just before p
            in BST order.
        '''
        self._validate(p)
        # If p has a left child, search for last position
        if self.left(p) is not None:
            return self._subtree_last_position(self.left(p))
        # If p does not have a left child, walk upward
        ## in tree until ancestor is a right child
        ## If p.key() is lowest, returns None.
        else:
            cur_pos = p
            cur_pos_parent = self.parent(cur_pos)
            while (cur_pos_parent is not None) and (cur_pos==self.left(cur_pos_parent)):
                cur_pos = cur_pos_parent
                cur_pos_parent = self.parent(cur_pos)
            return cur_pos_parent
    
    def after(self, p):
        '''
            Return Position right after p
            in BST order.
        '''
        self._validate(p)
        # If p has a left child, search for last position
        if self.right(p) is not None:
            return self._subtree_first_position(self.right(p))
        # If p does not have a left child, walk upward
        ## in tree until ancestor is a right child
        ## If p.key() is lowest, returns None.
        else:
            cur_pos = p
            cur_pos_parent = self.parent(cur_pos)
            while (cur_pos_parent is not None) and (cur_pos==self.right(cur_pos_parent)):
                cur_pos = cur_pos_parent
                cur_pos_parent = self.parent(cur_pos)
            return cur_pos_parent

In [8]:
myTree = BSTMap()

In [9]:
myTree["A"] = 2

In [12]:
myTree["A"]

3

In [11]:
myTree["A"] = 3